In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import plotly.express as px
import unidecode

In [2]:
from formula_one import FormulaOne

In [3]:
f1 = FormulaOne()
f1

Data
    Races
        (145, 7)
    Circuits
        (27, 7)
    Results
        (2967, 11)
    Drivers
        (51, 7)
    Constructors
        (19, 4)

In [4]:
f1.update()

'Data is up to date. | Most Recent RaceID: `202107`'

In [5]:
f1

Data
    Races
        (145, 7)
    Circuits
        (27, 7)
    Results
        (2967, 11)
    Drivers
        (51, 7)
    Constructors
        (19, 4)

In [6]:
fantasy_rosters = pd.read_csv('./draft/fantasy_rosters.csv', index_col=0)
fantasy_rosters

,OverallPick,TeamPick,Team,DriverID,LastName,FirstName,Car,Nationality
0,1,1,Carlaniel & the Romeos,norris,Norris,Lando,MCLAREN MERCEDES,GBR
1,2,1,Scuderia Spaghetti,hamilton,Hamilton,Lewis,MERCEDES,GBR
2,3,1,3 Orange Whips,max_verstappen,Verstappen,Max,RED BULL RACING HONDA,NED
3,4,1,Deep Fried,bottas,Bottas,Valtteri,MERCEDES,FIN
4,5,2,Deep Fried,perez,Perez,Sergio,RED BULL RACING HONDA,MEX
5,6,2,3 Orange Whips,gasly,Gasly,Pierre,ALPHATAURI HONDA,FRA
6,7,2,Scuderia Spaghetti,leclerc,Leclerc,Charles,FERRARI,MON
7,8,2,Carlaniel & the Romeos,sainz,Sainz,Carlos,FERRARI,ESP
8,9,3,Carlaniel & the Romeos,ricciardo,Ricciardo,Daniel,MCLAREN MERCEDES,AUS
9,10,3,Scuderia Spaghetti,russell,Russell,George,WILLIAMS MERCEDES,GBR


In [7]:
TEAM_NAMES_LST = [
    '3 Orange Whips',
    'Deep Fried',
    'Scuderia Spaghetti',
    'Carlaniel & the Romeos'
]
R = (
    fantasy_rosters.drop(['LastName', 'FirstName'], axis=1)
    .sort_values(['Team', 'TeamPick'])
    .merge(f1.drivers.drop(['Nationality', 'URL'], axis=1), on='DriverID')
    .drop(['DriverID'], axis=1)
    .loc[:, ['Team', 'FirstName', 'LastName', 
             'Car', 'DOB', 'TeamPick', 'OverallPick']]
)
rosters = [R.loc[R['Team'] == t]for t in TEAM_NAMES_LST]
rosters[0]

,Team,FirstName,LastName,Car,DOB,TeamPick,OverallPick
0,3 Orange Whips,Max,Verstappen,RED BULL RACING HONDA,1997-09-30,1,3
1,3 Orange Whips,Pierre,Gasly,ALPHATAURI HONDA,1996-02-07,2,6
2,3 Orange Whips,Fernando,Alonso,ALPINE RENAULT,1981-07-29,3,11
3,3 Orange Whips,Lance,Stroll,ASTON MARTIN MERCEDES,1998-10-29,4,14
4,3 Orange Whips,Nicholas,Latifi,WILLIAMS MERCEDES,1995-06-29,5,19


In [50]:
driver_score = (
    f1.results[f1.results['RaceID'] > 202100]
    .merge(f1.races[['RaceID', 'Round']])
    .loc[:, ['Round', 'Points', 'DriverID']]
    .sort_values(['DriverID', 'Round'])
)
for driver in driver_score['DriverID'].unique():
    cumulative = (
        driver_score
        .loc[driver_score['DriverID'] == driver]['Points']
        .cumsum()
    )
    driver_score.loc[driver_score['DriverID'] == driver, 'CumSum'] = cumulative
    
for rnd in driver_score['Round'].unique():
    rnd_zscores = stats.zscore(
        driver_score.loc[driver_score['Round'] == rnd]['CumSum']
    )
    driver_score.loc[driver_score['Round'] == rnd, 'CumZScore'] = rnd_zscores
    
driver_score.head(20)

,Round,Points,DriverID,CumSum,CumZScore
18,1,0,alonso,0.0,-0.703934
29,2,1,alonso,1.0,-0.680903
47,3,4,alonso,5.0,-0.512438
76,4,0,alonso,5.0,-0.569574
92,5,0,alonso,5.0,-0.652638
105,6,8,alonso,13.0,-0.538600
127,7,4,alonso,17.0,-0.475953
2,1,16,bottas,16.0,1.504486
37,2,0,bottas,16.0,0.429265
42,3,16,bottas,32.0,0.830846


In [51]:
px.line(x='Round', y='CumZScore', line_group='DriverID', data_frame=driver_score)

In [10]:
(f1.results[f1.results['RaceID'] > 202100]
 .merge(fantasy_rosters, on='DriverID')
 .groupby('Team').sum()[['Points']]
 .sort_values('Points', ascending=False)
 .reset_index()
)

,Team,Points
0,3 Orange Whips,195
1,Deep Fried,185
2,Scuderia Spaghetti,179
3,Carlaniel & the Romeos,154


In [11]:
pts_per_round = (
    f1.results.loc[f1.results['RaceID'] > 202100]
    .merge(fantasy_rosters[['Team', 'DriverID']], 
           on='DriverID')
    .merge(f1.races[['RaceID', 'Round']])
    .sort_values(['RaceID', 'Position'])
    .loc[:, ['Points', 'Team', 'Round']]
    .groupby(['Round', 'Team']).sum()
    .reset_index()
)

pts_per_round

,Round,Team,Points
0,1,3 Orange Whips,19
1,1,Carlaniel & the Romeos,22
2,1,Deep Fried,26
3,1,Scuderia Spaghetti,35
4,2,3 Orange Whips,36
5,2,Carlaniel & the Romeos,33
6,2,Deep Fried,2
7,2,Scuderia Spaghetti,31
8,3,3 Orange Whips,23
9,3,Carlaniel & the Romeos,12


In [12]:
cumulative_points = (
    pts_per_round
    .groupby(['Team', 'Round']).sum()
    .groupby('Team').cumsum()
    .reset_index()
)
cumulative_points

,Team,Round,Points
0,3 Orange Whips,1,19
1,3 Orange Whips,2,55
2,3 Orange Whips,3,78
3,3 Orange Whips,4,98
4,3 Orange Whips,5,135
5,3 Orange Whips,6,158
6,3 Orange Whips,7,195
7,Carlaniel & the Romeos,1,22
8,Carlaniel & the Romeos,2,55
9,Carlaniel & the Romeos,3,67


In [13]:
px.line(x='Round', y='Points', color='Team', 
        data_frame=cumulative_points, 
        title='Cumulative Points - 2021 Season')

In [14]:
px.bar(x='Round', y='Points', color='Team', 
       data_frame=pts_per_round,
      title='Points by Round')

In [15]:
TEAM_NAME = 'Deep Fried'

team_results = (f1.results.loc[f1.results['RaceID'] > 202100]
 .merge(fantasy_rosters
        .loc[fantasy_rosters['Team'] == TEAM_NAME][['Team', 'DriverID']], 
        on='DriverID')
 .merge(f1.races[['RaceID', 'Round']],
        on='RaceID')
 .merge(f1.drivers[['DriverID', 'Code']], on='DriverID')
)
team_results.head(14)

,RaceID,Position,Points,DriverID,ConstructorID,Grid,Laps,Status,Time,FastestLapTime,FastestLapSpeed,Team,Round,Code
0,202101,3,16,bottas,mercedes,3,56,Finished,+37.383,1:32.090,211.566,Deep Fried,1,BOT
1,202102,18,0,bottas,mercedes,8,30,Collision,NaN,1:28.485,199.721,Deep Fried,2,BOT
2,202103,3,16,bottas,mercedes,1,66,Finished,+33.530,1:19.865,209.738,Deep Fried,3,BOT
3,202104,3,15,bottas,mercedes,3,66,Finished,+26.610,1:19.430,211.884,Deep Fried,4,BOT
4,202105,19,0,bottas,mercedes,3,29,Wheel nut,NaN,1:15.706,158.682,Deep Fried,5,BOT
5,202106,12,0,bottas,mercedes,10,51,Finished,+11.264,1:45.665,204.521,Deep Fried,6,BOT
6,202107,4,12,bottas,mercedes,3,53,Finished,+14.618,1:36.960,216.905,Deep Fried,7,BOT
7,202101,5,10,perez,red_bull,0,56,Finished,+52.047,1:33.970,207.334,Deep Fried,1,PER
8,202102,11,0,perez,red_bull,2,63,Finished,+67.151,1:19.396,222.585,Deep Fried,2,PER
9,202103,4,12,perez,red_bull,4,66,Finished,+39.735,1:20.643,207.715,Deep Fried,3,PER


In [16]:
px.bar(x='Round', y='Points', color='Code', 
       data_frame=team_results, title=f'{TEAM_NAME} - Points by Driver')

In [17]:
# with plt.style.context(['ggplot', 'seaborn-talk']):
#     fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12, 8))
#     sns.lineplot(x='Round', y='Points', hue='Team', data=cumulative_points, ax=ax1)
#     sns.barplot(x='Round', y='Points', hue='Team', data=pts_per_round, ax=ax2)
#     fig.tight_layout()